# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

In [7]:
# Imports
import os
import warnings
import tools
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional
from matplotlib import pyplot



# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Root CSV files directory
dirname = "./data/"  

# Constant frame count.
frames = 100


#Preparation Stage - Load data and normalize
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))
        
#Split data 60-20-20

features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size=0.40, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.50, random_state=42)

#Enumerate
def printCountDataSets(dataset):
    wortCounter = []
    #Liste mit einmaligen Labels erstellen
    labels = sorted(set(dataset), key=dataset.index)
    #Liste nochmal Alphabetisch sortieren
    labels = sorted(labels)
    for label in labels:
        wortCounter.append(0)
    for row in dataset:
        for i in range(len(labels)):
            if str(labels[i]).startswith(row):
                wortCounter[i] += 1
    for i in range(len(labels)):
        print(labels[i], ': ', wortCounter[i], end =";  ")
    print(' ')        
    
print('Amount Datasets by word total:')
printCountDataSets(labels)
print('')

print('Amount Datasets by word training:')
printCountDataSets(y_train)
print('')

print('Amount Datasets by word validiation:')
printCountDataSets(y_val)
print('')

print('Amount Datasets by word test:')
printCountDataSets(y_test)
print('')


# Display data distribution
print('Distribution of data:')
print("Amount total:", len(labels))
print("Amount training:", len(y_train))
print("Amount validiation:", len(y_val))
print("Amount test:", len(y_test))
print('')

#Tokenize (One Hot)
tokenizer = tools.tokenize(dirname)
print('Tokens:')
print(tokenizer.word_index)
print('')
with open('tokens_json.txt', 'w') as outfile:
    outfile.write(tokenizer.to_json())

encoded_train=tokenizer.texts_to_sequences([y_train])[0]
encoded_val=tokenizer.texts_to_sequences([y_val])[0]
encoded_test=tokenizer.texts_to_sequences([y_test])[0]

y_train = to_categorical(encoded_train)
y_val = to_categorical(encoded_val)
y_test = to_categorical(encoded_test)

print('Categories in OneHot anotation:')
print(y_train)
print('')
# Making numpy arrays
x_train=np.array(x_train)
y_train=np.array(y_train)
x_val=np.array(x_val)
y_val=np.array(y_val)
x_test=np.array(x_test)
y_test=np.array(y_test)

print('Dataset coordinate Values:')
print(x_train)
print('')

import winsound
def finished(num):
    frequency = 2000  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    for i in range(0, num):
        winsound.Beep(frequency, duration)
    

Amount Datasets by word total:
Computer :  57;  Deutschland :  65;  Haben :  68;  Hallo :  57;  Mainz :  65;  Software :  67;  Welt :  66;  du :  66;  ich :  66;  unser :  64;  zeigen :  69;   

Amount Datasets by word training:
Computer :  31;  Deutschland :  30;  Haben :  38;  Hallo :  34;  Mainz :  38;  Software :  46;  Welt :  43;  du :  42;  ich :  37;  unser :  43;  zeigen :  44;   

Amount Datasets by word validiation:
Computer :  13;  Deutschland :  15;  Haben :  11;  Hallo :  11;  Mainz :  10;  Software :  13;  Welt :  14;  du :  13;  ich :  19;  unser :  10;  zeigen :  13;   

Amount Datasets by word test:
Computer :  13;  Deutschland :  20;  Haben :  19;  Hallo :  12;  Mainz :  17;  Software :  8;  Welt :  9;  du :  11;  ich :  10;  unser :  11;  zeigen :  12;   

Distribution of data:
Amount total: 710
Amount training: 426
Amount validiation: 142
Amount test: 142

Tokens:
{'computer': 1, 'deutschland': 2, 'du': 3, 'haben': 4, 'hallo': 5, 'ich': 6, 'mainz': 7, 'software': 8,

## Training Stage
Configure the model and train it.

Metrics:
<div float="right">
    <img src="assets/accuracy.png" width="400"> 
    <img src="assets/precision_recall_formula.png" width="400">
</div>
<img src="assets/precision_recall.png" width="1000">

#### LSTM

In [ ]:

model = Sequential()
model.add(layers.LSTM(256, return_sequences=True,
               input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(layers.LSTM(64, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32))  # return a single vector of dimension 32
#model.add(layers.LSTM(16))  # return a single vector of dimension 32
model.add(layers.Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

### or
#### Bidirectional LSTM

In [ ]:
model = Sequential()
model.add(Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(100, 86)))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
history=model.fit(x_train,y_train,epochs=170,validation_data=(x_val,y_val),shuffle=False)

### or
#### Simple RNN

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(200, 42)))
model.add(Dense(64, activation="softmax")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(128, activation="linear")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(21))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

### or
#### <span style="color:blue"> Hyperparametertuned LSTM </span>
##### Here it is necessary to install the Keras-Tuner Module by executing:
#####  <span style="color:green"> via Conda:</span>
conda install -c conda-forge keras-tuner
#####  <span style="color:green"> for pip:</span>
pip install keras-tuner


In [5]:
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
from time import time, strftime


starttime= strftime("%Y_%m_%d_%H%M%S")
LOG_DIR = "C:\ML\Optimization_"f"{starttime}" #<-In Windows below Log_dir Path will maybe be too long for Windows to handle, so use a shorter path like this here
#LOG_DIR = "./Optimization_"f"{starttime}" # LOG_DIR holds json files with information and a model of each single trial

def build_model_old_lstm(hp):
    model = Sequential()
    
    model.add(layers.LSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64), #kerastuner will randomly choose a value for nodes between 128 and 256 in steps of 64
                            return_sequences=True,
                            input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True))
    
    model.add(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32)))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adam','RMSprop','SGD']),
                  optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Nadam','RMSprop']),
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model


def build_model_bdlstm(hp):
    model = Sequential()
    model.add(Bidirectional(layers.LSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64),
                                        return_sequences=True),
                                        input_shape=(100, 86)))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(layers.Bidirectional(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True)))
    
    model.add(layers.Bidirectional(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32))))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Nadam','RMSprop']),
                  optimizer=hp.Choice('optimizer',values=['Adam','SGD']),
                  metrics=['accuracy']) 
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model






###   <span style="color:red">Necesarry only in case of using Nvidia GPU  </span>

In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs: 1


# Different Keras-Tuner Approaches
### 1 - RandomSearch
Parameter of variables are ranomly used (number of layers, number of nodes) and "best" model is chosen.

In [ ]:
tuner  = RandomSearch(
    build_model_bdlstm,     #Function to use search in
    objective = "val_accuracy",  #Chooses "best model" looking for highest value of val_accuracy
    max_trials = 30,       # Number of different combinations tried Nodes and layers
    executions_per_trial = 1, 
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train,      #syntax just like in fit
                y= y_train,
            epochs=200,
            #batch_size=32,
            validation_data=(x_val,y_val),
            verbose=2, 
            use_multiprocessing=True,
            workers=4, )

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

finished(8)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100, 128)          77312     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 12)                780       
Total params: 218,124
Trainable params: 218,124
Non-trainable params: 0
_________________________________________________________________
Adam

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100, 512)          702464    
________________________

Epoch 63/200
426/426 - 1s - loss: 1.4928 - accuracy: 0.4906 - val_loss: 1.5883 - val_accuracy: 0.4085
Epoch 64/200
426/426 - 1s - loss: 1.4735 - accuracy: 0.4836 - val_loss: 1.4791 - val_accuracy: 0.4085
Epoch 65/200
426/426 - 1s - loss: 1.4316 - accuracy: 0.5047 - val_loss: 1.5029 - val_accuracy: 0.4648
Epoch 66/200
426/426 - 1s - loss: 1.4976 - accuracy: 0.4765 - val_loss: 1.4388 - val_accuracy: 0.4296
Epoch 67/200
426/426 - 1s - loss: 1.4161 - accuracy: 0.5047 - val_loss: 1.4936 - val_accuracy: 0.4366
Epoch 68/200
426/426 - 1s - loss: 1.4125 - accuracy: 0.5094 - val_loss: 1.5541 - val_accuracy: 0.4225
Epoch 69/200
426/426 - 1s - loss: 1.3961 - accuracy: 0.5047 - val_loss: 1.4455 - val_accuracy: 0.4366
Epoch 70/200
426/426 - 1s - loss: 1.4233 - accuracy: 0.4977 - val_loss: 1.4348 - val_accuracy: 0.4789
Epoch 71/200
426/426 - 1s - loss: 1.3519 - accuracy: 0.5211 - val_loss: 1.3884 - val_accuracy: 0.4930
Epoch 72/200
426/426 - 1s - loss: 1.3485 - accuracy: 0.5094 - val_loss: 1.4128 - v

Epoch 143/200
426/426 - 1s - loss: 0.7993 - accuracy: 0.7418 - val_loss: 0.9220 - val_accuracy: 0.6761
Epoch 144/200
426/426 - 1s - loss: 0.7398 - accuracy: 0.7418 - val_loss: 0.9209 - val_accuracy: 0.6690
Epoch 145/200
426/426 - 1s - loss: 0.7415 - accuracy: 0.7535 - val_loss: 0.9024 - val_accuracy: 0.6831
Epoch 146/200
426/426 - 1s - loss: 0.7331 - accuracy: 0.7606 - val_loss: 0.8024 - val_accuracy: 0.7324
Epoch 147/200
426/426 - 1s - loss: 0.8167 - accuracy: 0.7230 - val_loss: 1.3242 - val_accuracy: 0.4296
Epoch 148/200
426/426 - 1s - loss: 0.8590 - accuracy: 0.6878 - val_loss: 0.8825 - val_accuracy: 0.6761
Epoch 149/200
426/426 - 1s - loss: 0.9185 - accuracy: 0.6808 - val_loss: 1.6676 - val_accuracy: 0.4155
Epoch 150/200
426/426 - 1s - loss: 0.9163 - accuracy: 0.6737 - val_loss: 0.8058 - val_accuracy: 0.7113
Epoch 151/200
426/426 - 1s - loss: 0.7401 - accuracy: 0.7371 - val_loss: 0.9765 - val_accuracy: 0.6056
Epoch 152/200
426/426 - 1s - loss: 0.7014 - accuracy: 0.7817 - val_loss: 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100, 384)          428544    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 256)          525312    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               164352    
_________________________________________________________________
dense (Dense)                (None, 12)                1548      
Total params: 1,119,756
Trainable params: 1,119,756
Non-trainable params: 0
_________________________________________________________________
Adam

Train on 426 samples, validate on 142 samples
Epoch 1/200
426/426 - 9s - loss: 2.2712 - accuracy: 0.1268 - val_loss: 1.9037 - val_accuracy: 0.2817
Epoch 2/200
426/426 - 1s - loss: 1.8703 - accuracy: 0.2629 - val_loss: 1.7700 - val_accurac

Epoch 73/200
426/426 - 1s - loss: 0.2010 - accuracy: 0.9460 - val_loss: 0.9084 - val_accuracy: 0.7324
Epoch 74/200
426/426 - 1s - loss: 0.1910 - accuracy: 0.9484 - val_loss: 1.0098 - val_accuracy: 0.7113
Epoch 75/200
426/426 - 1s - loss: 0.2452 - accuracy: 0.9225 - val_loss: 0.9312 - val_accuracy: 0.6972
Epoch 76/200
426/426 - 1s - loss: 0.2521 - accuracy: 0.9131 - val_loss: 0.9518 - val_accuracy: 0.7254
Epoch 77/200
426/426 - 1s - loss: 0.1942 - accuracy: 0.9484 - val_loss: 1.0148 - val_accuracy: 0.7183
Epoch 78/200
426/426 - 1s - loss: 0.1696 - accuracy: 0.9577 - val_loss: 0.9897 - val_accuracy: 0.7324
Epoch 79/200
426/426 - 1s - loss: 0.1427 - accuracy: 0.9671 - val_loss: 0.9871 - val_accuracy: 0.7324
Epoch 80/200
426/426 - 1s - loss: 0.1373 - accuracy: 0.9577 - val_loss: 1.0456 - val_accuracy: 0.7183
Epoch 81/200
426/426 - 1s - loss: 0.1466 - accuracy: 0.9531 - val_loss: 0.9031 - val_accuracy: 0.7324
Epoch 82/200
426/426 - 1s - loss: 0.1592 - accuracy: 0.9413 - val_loss: 0.9178 - v

...

Epoch 151/200
426/426 - 1s - loss: 0.0053 - accuracy: 1.0000 - val_loss: 0.9611 - val_accuracy: 0.7606
Epoch 152/200
426/426 - 1s - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.9638 - val_accuracy: 0.7606
Epoch 153/200
426/426 - 1s - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.9658 - val_accuracy: 0.7606
Epoch 154/200
426/426 - 1s - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.9712 - val_accuracy: 0.7606
Epoch 155/200
426/426 - 1s - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.9705 - val_accuracy: 0.7606
Epoch 156/200
426/426 - 1s - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.9695 - val_accuracy: 0.7606
Epoch 157/200
426/426 - 1s - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.9746 - val_accuracy: 0.7606
Epoch 158/200
426/426 - 1s - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.9765 - val_accuracy: 0.7606
Epoch 159/200
426/426 - 1s - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.9789 - val_accuracy: 0.7606
Epoch 160/200
426/426 - 1s - loss: 0.0040 - accuracy: 1.0000 - val_loss: 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100, 384)          428544    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 256)          525312    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 256)          394240    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 384)          689664    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                106752    
_________________________________________________________________
dense (Dense)                (None, 12)                780       
Total params: 2,145,292
Trainable params: 2,145,292
Non-trainable params: 0
______________________________________________

Epoch 70/200
426/426 - 1s - loss: 0.8241 - accuracy: 0.7230 - val_loss: 0.9996 - val_accuracy: 0.6690
Epoch 71/200
426/426 - 1s - loss: 0.6418 - accuracy: 0.7934 - val_loss: 1.1593 - val_accuracy: 0.6127
Epoch 72/200
426/426 - 1s - loss: 0.7166 - accuracy: 0.7324 - val_loss: 1.0172 - val_accuracy: 0.6549
Epoch 73/200
426/426 - 1s - loss: 0.5978 - accuracy: 0.7887 - val_loss: 1.0774 - val_accuracy: 0.6268
Epoch 74/200
426/426 - 1s - loss: 0.5699 - accuracy: 0.7981 - val_loss: 1.0268 - val_accuracy: 0.6831
Epoch 75/200
426/426 - 1s - loss: 0.4746 - accuracy: 0.8333 - val_loss: 0.9583 - val_accuracy: 0.7183
Epoch 76/200
426/426 - 1s - loss: 0.5462 - accuracy: 0.8146 - val_loss: 1.1583 - val_accuracy: 0.6408
Epoch 77/200
426/426 - 1s - loss: 0.5835 - accuracy: 0.8192 - val_loss: 0.9942 - val_accuracy: 0.6831
Epoch 78/200
426/426 - 1s - loss: 0.5993 - accuracy: 0.8005 - val_loss: 1.1172 - val_accuracy: 0.6690
Epoch 79/200
426/426 - 1s - loss: 0.5185 - accuracy: 0.8169 - val_loss: 0.9289 - v

Epoch 150/200
426/426 - 1s - loss: 0.2620 - accuracy: 0.9178 - val_loss: 0.9690 - val_accuracy: 0.7254
Epoch 151/200
426/426 - 1s - loss: 0.1743 - accuracy: 0.9577 - val_loss: 1.1210 - val_accuracy: 0.6901
Epoch 152/200
426/426 - 1s - loss: 0.2306 - accuracy: 0.9155 - val_loss: 1.0257 - val_accuracy: 0.7183
Epoch 153/200
426/426 - 1s - loss: 0.2279 - accuracy: 0.9272 - val_loss: 0.9307 - val_accuracy: 0.7535
Epoch 154/200
426/426 - 1s - loss: 0.2119 - accuracy: 0.9343 - val_loss: 0.9944 - val_accuracy: 0.7394
Epoch 155/200
426/426 - 1s - loss: 0.2062 - accuracy: 0.9343 - val_loss: 0.9340 - val_accuracy: 0.7606
Epoch 156/200
426/426 - 1s - loss: 0.2067 - accuracy: 0.9319 - val_loss: 1.1851 - val_accuracy: 0.6831
Epoch 157/200
426/426 - 1s - loss: 0.2346 - accuracy: 0.9202 - val_loss: 1.0230 - val_accuracy: 0.7183
Epoch 158/200
426/426 - 1s - loss: 0.1817 - accuracy: 0.9343 - val_loss: 0.9846 - val_accuracy: 0.7535
Epoch 159/200
426/426 - 1s - loss: 0.1264 - accuracy: 0.9648 - val_loss: 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100, 256)          220160    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 384)          689664    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 384)          886272    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                106752    
_________________________________________________________________
dense (Dense)                (None, 12)                780       
Total params: 1,903,628
Trainable params: 1,903,628
Non-trainable params: 0
_________________________________________________________________
SGD

Train on 426 samples, validate on 142 samples
Epoch 1/200
426/426 - 12s - loss: 2.4800 - accuracy: 0.1150 -

### 2 - Hyperband
Variation of RandomSearch http://jmlr.org/papers/volume18/16-558/16-558.pdf

In [ ]:
tuner  = Hyperband(
    build_model,
    objective = "val_accuracy",
    hyperband_iterations=2,
    max_epochs=150,
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train, 
            y= y_train,
            batch_size=32,
            validation_data=(x_val,y_val))

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

finished(8)

In [ ]:
#Laut Randomsearch bestes Model am 23.06.2020

model = Sequential()
model.add(layers.LSTM(128, return_sequences=True,
               input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(layers.LSTM(64, return_sequences=True)) 
model.add(layers.LSTM(96))  
model.add(layers.Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
model.summary()

#history=model.fit(x_train,y_train,epochs=170,validation_data=(x_val,y_val),shuffle=False)

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

## Export tuner object into pickle file
so it can be used in other scripts

In [ ]:
import pickle

with open(f"tuner_"f"{starttime}.pkl", "wb") as f:
    pickle.dump(tuner, f)
    

## Get best Trial from Tuner Object

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
bestmodel= tuner.hypermodel.build(best_hp)

bestmodel.summary()


In [ ]:
#tmp_chekpoints= "tmp\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"
tmp_chekpoints= "C:\\ML\\checkpoints\\tmp\\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"

#csv_log = tf.keras.callbacks.CSVLogger("log.csv", separator=',', append=False)
csv_log = tf.keras.callbacks.CSVLogger("C:\ML\logs\log.csv", separator=',', append=False)

#tb = tf.keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
tb = tf.keras.callbacks.TensorBoard(log_dir='C:\ML\logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=20, verbose=0, mode='max', baseline=None, restore_best_weights=True)
chk= tf.keras.callbacks.ModelCheckpoint(tmp_chekpoints, monitor='val_accuracy', verbose=0, save_best_only=False, save_weights_only=False, mode='max', save_freq='epoch')


history = model.fit(x_train,y_train,epochs=200,batch_size=32, validation_data=(x_val,y_val),shuffle=False, verbose=2, callbacks=[csv_log, chk])

### Diagnostic Plots

The training history of your LSTM models can be used to diagnose the behavior of your model.

You can plot the performance of your model using the Matplotlib library. For example, you can plot training loss vs test loss as follows:

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.savefig("C:/ML/loss"f"{starttime}.png")
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='lower right')
pyplot.savefig("C:/ML/accuracy_"f"{starttime}.png")
pyplot.show()

#### Underfit Example
Running this example produces a plot of train and validation loss showing the characteristic of an underfit model. In this case, performance may be improved by increasing the number of training epochs.


<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model.png" width="400">


Running this example shows the characteristic of an underfit model that appears under-provisioned.
In this case, performance may be improved by increasing the capacity of the model, such as the number of memory cells in a hidden layer or number of hidden layers.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model-via-Status.png" width="400">

#### Good Fit Example
Running the example creates a line plot showing the train and validation loss meeting.
Ideally, we would like to see model performance like this if possible, although this may not be possible on challenging problems with a lot of data.

<img src="assets/Diagnostic-Line-Plot-Showing-a-Good-Fit-for-a-Model.png" width="400">

#### Overfit Example
Running this example creates a plot showing the characteristic inflection point in validation loss of an overfit model.
This may be a sign of too many training epochs.
In this case, the model training could be stopped at the inflection point. Alternately, the number of training examples could be increased.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Overfit-Model.png" width="400">

### Evaluate

In [ ]:
#model = tf.keras.models.load_model('./tmp/epoch49-0.90-0.39.hdf5')


#bestmodel.evaluate(x=x_test, y=y_test, verbose=2)
model.evaluate(x=x_test, y=y_test, verbose=2)


### Save model

In [ ]:
bestmodel.save("sign_lang_recognition_tuned.h5")